# IMPORTS

In [36]:
# from grobid_client.grobid_client import GrobidClient
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import os
import re
import sys
import gensim
from gensim import corpora, models, similarities
import pickle

import os
from stop_words import get_stop_words
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
import numpy as np
# Now let's do topic modeling using LDA
from sklearn.decomposition import LatentDirichletAllocation


# si no hace bien los imports de utilsdescomenta esta linea
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from utils import remove_files, get_abstract, cosine

# GET papers

In [37]:
# Directorio donde se encuentran los archivos XML
xml_dir = os.path.join(parent_dir, "output")
# papers_dir = os.path.join(parent_dir, "papers")

# remove_files(xml_dir)

# client = GrobidClient(config_path="./config.json")
# client.process("processFulltextDocument", papers_dir, output=xml_dir, consolidate_citations=True, tei_coordinates=True, n=20)

# Lista para almacenar los resúmenes
papers = {}

# Procesar cada archivo XML en el directorio
for file in os.listdir(xml_dir):
    if file.endswith(".xml"):  # Verificar que el archivo sea XML
        file_path = os.path.join(xml_dir, file)
        file_name = os.path.basename(file_path)[:-15]
        tree = ET.parse(file_path)
        root = tree.getroot()
        #verificamos si el paper esta en nuestro diccionario y si no lo esta creamos una entrada para el
        if file_name not in papers:
            papers[file_name] = {}

        #get abstract
        abstract = get_abstract(root)
        papers[file_name]['abstract'] = abstract

        #get authors
        ns = {'tei': 'http://www.tei-c.org/ns/1.0'}
        autor_loc = root.findall('.//tei:sourceDesc', ns)
        acknowledgePerson = root.findall('.//tei:author', ns)
        acknowledgeOrg = root.findall('.//tei:author', ns)
        # Obtener los nombres completos de los autores
        nombres_completos = []
        for loc in autor_loc:
            autors = loc.findall('.//tei:persName', ns)
            # Encontrar el elemento forename dentro de author
                    # Verificar si se encontraron los elementos forename y surname
            for autor in autors:
                forename_elem = autor.findall('.//tei:forename', ns)
                # Encontrar el elemento surname dentro de author
                surname_elem = autor.findall('.//tei:surname', ns)
                
                try:
                    # Obtener el texto de los elementos forename y surname
                    nombre = forename_elem[0].text
                    apellido = surname_elem[0].text
                    nombre_completo = f"{nombre} {apellido}"
                except:
                    # Si no se encontraron los elementos, asignar NA a nombre y apellido
                    # print('Missing name and/or surname. Cant get full name.')
                    pass
                nombres_completos.append(nombre_completo)
            papers[file_name]['written_by'] = nombres_completos
        
        # Obtener la fecha de publicación
        try:
            published_date = root.find('.//tei:fileDesc/tei:publicationStmt/tei:date', ns).text
            papers[file_name]['published_date'] = published_date
        except:
            pass

         # Obtener el DOI (id) del documento
        # print(file_name)
        doi = root.find('.//tei:fileDesc/tei:sourceDesc/tei:biblStruct/tei:idno[@type="MD5"]', ns).text
        papers[file_name]['id'] = doi

        nombres_completosRef = []
        for autor in acknowledgePerson:
            # Encontrar el elemento forename dentro de author
            forename_elem = autor.find('.//tei:forename', ns)
            # Encontrar el elemento surname dentro de author
            surname_elem = autor.find('.//tei:surname', ns)
            
            # Verificar si se encontraron los elementos forename y surname
            try:
                # Obtener el texto de los elementos forename y surname
                nombre = forename_elem.text
                apellido = surname_elem.text
                nombre_completoRef = f"{nombre} {apellido}"
            except:
                # Si no se encontraron los elementos, asignar NA a nombre y apellido
                # print('Missing name and/or surname. Cant get full name.')
                pass
            if nombre_completoRef not in nombres_completosRef and nombre_completoRef not in nombres_completos:
                nombres_completosRef.append(nombre_completoRef)
        papers[file_name]['acknowledgePerson'] = nombres_completosRef
        
        nombres_org = []
        for org in acknowledgeOrg:
            # Encontrar el elemento forename dentro de autho
            forename_elem = org.find('.//tei:orgName[@type="institution"]', ns)

            # Verificar si se encontraron los elementos forename y surname
            try:
                # Obtener el texto de los elementos forename y surname
                nombre = forename_elem.text
                nombre_org = nombre
            except:
                # Si no se encontraron los elementos, asignar NA a nombre y apellido
                # print('Missing name and/or surname. Cant get full name.')
                pass
            if nombre_org != None and nombre_org not in nombres_org:
                nombres_org.append(nombre_org)
        papers[file_name]['acknowledgeOrg'] = nombres_org


print(f'Numero de documentos: {len(papers)}')

Numero de documentos: 19


In [38]:
papers

{'11621ijccsa02': {'abstract': 'With recent advances in technology, internet has drastically changed the computing world from the concept of parallel computing to distributed computing to grid computing and now to cloud computing. The evolution of cloud computing over the past few years is potentially one of the major advances in the history of computing. Unfortunately, many banks are still hesitant to adopt cloud technology. New credit unions wanting to achieve greater business agility, cloud technology enables organizations to respond instantly to changing market conditions, leveraging data and applied analytics to achieve customer experience and operational productivity benefits. As a result, cloud computing comes in to provide a solution to such challenges making banking a reliable and trustworthy service. This paper aims at cloud computing strategy, impact in banking and financial institutions and discusses the significant reliance of cloud computing.',
  'written_by': ['Prudhvi P

In [39]:
papers_list = [paper['abstract'] for paper in papers.values()]

# SIMILARITY


## BERT

In [40]:
# USING TRANSFORMERS
threshold = 0.6
# If we want to improve the similarity and use a word embeddings approach, we may use sentence transformers. This may take a while:
sbert_model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)
sentence_embeddings = sbert_model.encode(papers_list)
for i in range(len(sentence_embeddings)):
    for j in range(i + 1, len(sentence_embeddings)):
        sim = cosine(sentence_embeddings[i], sentence_embeddings[j])
        # print(
        #     f"La similitud entre el documento {list(papers.keys())[i]} y el documento {list(papers.keys())[j]} es sim {sim}"
        # )
        if sim > threshold:
            if 'has_Similarity' not in papers[list(papers.keys())[i]]:
                papers[list(papers.keys())[i]]['has_Similarity'] = {}

            if 'has_Similarity' not in papers[list(papers.keys())[j]]:
                papers[list(papers.keys())[j]]['has_Similarity'] = {}

            papers[list(papers.keys())[i]]['has_Similarity'][list(papers.keys())[j]] = sim
            papers[list(papers.keys())[j]]['has_Similarity'][list(papers.keys())[i]] = sim

c:\Users\nicov\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# TOPIC MODELLING

## LDA

In [41]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
# let's do a countvectorizer now. This is different from TF-IDF
# we are only creating 2 topics

from nltk.corpus import stopwords
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

tokens = [
    [
        word
        for word in paper_info['abstract'].lower().split()
        if word.isalpha() and word not in stop_words
    ]
    for paper_info in papers.values()
]


# Filtrar las stop words

# Unir los tokens filtrados de nuevo en un texto
tokens_flatten = [' '.join(lista) for lista in tokens]


In [42]:
# let's do a countvectorizer now. This is different from TF-IDF
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(tokens_flatten)
# we are only creating 2 topics
lda = LatentDirichletAllocation(n_components=10, random_state=0)
lda.fit(X)

LatentDirichletAllocation(random_state=0)

In [43]:
feature_names = count_vectorizer.get_feature_names_out()
for topic_id, topic in enumerate(lda.components_):
    print(f"Topic {topic_id}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-6:-1]]))

Topic 0:
computing cloud big data processing
Topic 1:
training may significant depending must
Topic 2:
local image features loss deep
Topic 3:
cloud computing model smbs significant
Topic 4:
intelligence artificial generated images may
Topic 5:
computer deep learning knowledge learn
Topic 6:
artificial intelligence need potential ethical
Topic 7:
cloud based computing visual security
Topic 8:
time intelligence series artificial features
Topic 9:
provides infrastructure associated risks models


In [44]:
#Now let's see the probability of one of the sentence to belong to each topic

for name, paper_info in papers.items():
    abstract = paper_info['abstract']
    new_doc_bow = count_vectorizer.transform([abstract])
    # Calcular la distribución de temas para el nuevo documento
    topic_distribution = lda.transform(new_doc_bow)
    # print(f"Topic distribution for {name}:")
    paper_info['topics_prob'] = {}
    for topic_idx, topic_prob in enumerate(topic_distribution[0]):
        topic = lda.components_[topic_idx]
        topic  = " ".join([feature_names[i] for i in topic.argsort()[:-6:-1]])
        paper_info['topics_prob'][topic] =topic_prob
        # print(f"Topic {topic_idx}: {topic_prob:.4f}")


In [45]:
papers

{'11621ijccsa02': {'abstract': 'With recent advances in technology, internet has drastically changed the computing world from the concept of parallel computing to distributed computing to grid computing and now to cloud computing. The evolution of cloud computing over the past few years is potentially one of the major advances in the history of computing. Unfortunately, many banks are still hesitant to adopt cloud technology. New credit unions wanting to achieve greater business agility, cloud technology enables organizations to respond instantly to changing market conditions, leveraging data and applied analytics to achieve customer experience and operational productivity benefits. As a result, cloud computing comes in to provide a solution to such challenges making banking a reliable and trustworthy service. This paper aims at cloud computing strategy, impact in banking and financial institutions and discusses the significant reliance of cloud computing.',
  'written_by': ['Prudhvi P

# SAVE DICT

In [46]:

# Guardar el diccionario en un archivo usando pickle
with open('papers_metadata.pickle', 'wb') as f:
    pickle.dump(papers, f)
